# How to use multiple memory classes in the same chain
It is also possible to use multiple memory classes in the same chain. To combine multiple memory classes, we can initialize the `CombinedMemory` class, and then use that.

In [76]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory, CombinedMemory, ConversationSummaryMemory

from datetime import datetime

import faiss
from langchain.chains import ConversationChain
from langchain.docstore import InMemoryDocstore
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.llms import OpenAI
from langchain.memory import VectorStoreRetrieverMemory
from langchain.prompts import PromptTemplate
from langchain.vectorstores import FAISS

embeddings = OpenAIEmbeddings()
air_vectorstore = FAISS.load_local("airstack_faiss_index", embeddings)
retriver = air_vectorstore.as_retriever(search_kwargs=dict(k=1))
memory_air = VectorStoreRetrieverMemory(retriever=retriver, memory_key="airstack")

embeddings_uni = OpenAIEmbeddings()
uni_vectorstore = FAISS.load_local("airstack_faiss_index", embeddings_uni)
retriver_uni = uni_vectorstore.as_retriever(search_kwargs=dict(k=1))
memory_uni = VectorStoreRetrieverMemory(retriever=retriver_uni, memory_key="uniswap")


conv_memory = ConversationBufferMemory(
    memory_key="chat_history_lines",
    input_key="input"
)

summary_memory = ConversationSummaryMemory(llm=OpenAI(temperature=0, model_name='gpt-4', max_tokens=3999), input_key="input")
# Combined
memory = CombinedMemory(memories=[conv_memory, summary_memory, memory_air, memory_uni])
_DEFAULT_TEMPLATE = """

you are are designed to help the developer, when a user asks a question, you should write the code to solve the problem and explain it as much as possible

airstack docs
{airstack}

uniswap docs
{uniswap}

Summary of conversation:
{history}
Current conversation:
{chat_history_lines}
Human: {input}
AI:"""
PROMPT = PromptTemplate(
    input_variables=["history", "input", "chat_history_lines", "airstack", "uniswap"], template=_DEFAULT_TEMPLATE
)
llm = OpenAI(temperature=0)
conversation = ConversationChain(
    llm=llm, 
    verbose=True, 
    memory=memory,
    prompt=PROMPT
)

In [77]:
conversation.run("how do i stake my eth?")



> Entering new ConversationChain chain...
Prompt after formatting:


you are are designed to help the developer, when a user asks a question, you should write the code to solve the problem and explain it as much as possible

airstack docs
Wallet API Examples
For ENS dwr.eth, get Farcaster name, Farcaster account details, connected address, and all token balances and images
query
 
identity
 
{
 
Wallet
(
input
:
 
{
identity
:
 
"dwr.eth"
,
 
blockchain
:
 
ethereum
})
 
{
 
socials
 
{
 
dappName
 
profileName
 
profileCreatedAtBlockTimestamp
 
userAssociatedAddresses
 
}
 
tokenBalances
 
{
 
tokenAddress
 
amount
 
tokenId
 
tokenType
 
tokenNfts
 
{
 
contentValue
 
{
 
image
 
{
 
original
 
}
 
}
 
token
 
{
 
name
 
}
 
}
 
}
 
}
}
For stani.lens show all his web3 socials and NFTs on Polygon
query
 
staniLensSocialsAndNFTs
 
{
 
Wallet
(
input
:
 
{
identity
:
 
"stani.lens"
,
 
blockchain
:
 
polygon
})
 
{
 
socials
 
{
 
dappName
 
profileName
 
}
 
tokenBalances
 
{
 
toke

' To stake your ETH, you will need to use a staking platform such as Uniswap. Uniswap provides a variety of staking options, including staking ETH, ERC-20 tokens, and other assets. To get started, you will need to create an account on Uniswap and deposit your ETH into the platform. Once your ETH is deposited, you can select the staking option that best suits your needs. You can find more information about staking on Uniswap in their documentation.'